In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX
from csv import writer
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import csv_to_sqlite
from functions import *

In [2]:
#Change working directory to navicage files
os.chdir("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")

In [3]:
#Function to extract 1045 sheet name from xlsx files

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file, data_only=True)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form", "analysis", "AB"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [4]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(somefile):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(somefile)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        try: #when sheet is nonetype (doesn't exist) we need to skip it
            for row in sheet.iter_rows():
                    for cell in row:
                        heading = str(cell.value).lower()
                        wordsInHeading=heading.split()
                        match =  all(item in wordsInHeading for item in wordlist)
                        if match==True:
                            coord1=cell.coordinate
                            coordlist.append(coord1)
                            break
        except:
            return("FAIL")
            continue
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])

        dflist.append(df)

    return(dflist)

  

In [5]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [6]:
#Function to clean raw data, getting real values, adding columns and hospital names, and removing completely empty rows.
def getMaster(somefile):
        master=pd.DataFrame(columns = ['Service', 'CPT',"AvCharge", 'Hospital', "Service Type"])
        hospital = os.path.basename(os.path.dirname(somefile))
        rawlist=getdata_xlsx(somefile)
        for i in range(len(rawlist)):
                try:
                        df=rawlist[i].applymap(convertvalue)
                        df=df.dropna(how='all')
                        df.columns = ['Service', 'CPT',"AvCharge"]
                        df["Service Type"]=df.iloc[0,0]
                        df["Hospital"] = hospital
                        master=master.append(df[1:])
                except:
                        continue
        return(master)

In [7]:
#Function to get a list of files in a directory (code borrowed from StackOverflow)

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries99213/G0463
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [8]:
#Create Column Labels
alldata=pd.DataFrame(columns = ['Service', 'CPT',"AvCharge", 'Hospital', "ServiceType"])
with open('/home/ygael/Projects/healthcare/data/csv/data.csv', 'a') as f:
    alldata.to_csv(f, header=True)

In [9]:
#Collect data in CSV file
hitcounter=0
filelist=getListOfFiles("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")
for filename in filelist:
    if filename[-4:]=="xlsx":
        try:
            df=getMaster(filename)
            df.replace({'99213/G0463': 99213, 'G0463': 99213,'-': "",  'N/A': ""}, inplace=True)
            with open('/home/ygael/Projects/healthcare/data/csv/data.csv', 'a') as f:
                 df.to_csv(f, header=False)
            print(f"Total extracted: {hitcounter}")
            hitcounter+=1
        except:
            print (f"File skipped: {filename}")
            continue

Total extracted: 0
Total extracted: 1
Total extracted: 2
Total extracted: 3
Total extracted: 4
Total extracted: 5
Total extracted: 6
Total extracted: 7
Total extracted: 8
Total extracted: 9
Total extracted: 10
Total extracted: 11
Total extracted: 12
Total extracted: 13
Total extracted: 14
Total extracted: 15
Total extracted: 16
Total extracted: 17
Total extracted: 18
Total extracted: 19
Total extracted: 20
Total extracted: 21
Total extracted: 22
Total extracted: 23
Total extracted: 24
Total extracted: 25
Total extracted: 26
Total extracted: 27
Total extracted: 28
Total extracted: 29
Total extracted: 30
Total extracted: 31
Total extracted: 32
Total extracted: 33
/home/ygael/anaconda3/envs/DataViz/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
Total extracted: 34
Total extracted: 35
/home/ygael/anaconda3/envs/DataViz/lib/python3.8/site-pa